# Part 1 Building our RAG AI Tutor - Using LLMs

In [1]:
import enum
import logging as log
import os
import pprint
import requests

In [2]:
UrlCompletions = "https://api.openai.com/v1/chat/completions"

In [3]:
log.basicConfig(level=log.DEBUG, format='%(asctime)s [%(levelname)5s] - %(message)s',
                  datefmt='%H:%M:%S')

In [4]:
class FinishReason(enum.Enum):
    Stop=0,
    LengthExceeded=1

def parseFinishReason( s: str):
    if 'stop'==s:
        return FinishReason.Stop        
    elif 'length'==s:       
        return FinishReason.LengthExceeded
    raise ValueError(f'Invalid finish reason:{s}')

In [5]:
class LlmResponse():
  def __init__(self, content:str, 
               numPromptTokens: int, 
               numCompletionTokens: int, 
               finishReason: FinishReason):
    self._content= content
    self._numPromptTokens= numPromptTokens
    self._numCompletionTokens= numCompletionTokens
    self._finishReason= finishReason

  @property
  def content(self) -> str: return self._content

  @property
  def numPromptTokens(self) -> int: return self._numPromptTokens

  @property
  def numCompletionTokens(self) -> int: return self._numCompletionTokens

  @property
  def finishReason(self) -> FinishReason: return self._finishReason

  def __str__(self):
      return f'content:\"{ self._content}\"\nnumPromptTokens:{ self._numPromptTokens}\nnumCompletionTokens:{ self._numCompletionTokens}\nfinishReason:{ self._finishReason.name}'

In [6]:
def promptTranslationGetResponse( userPrompt: str, systemRole='You are a helpful assistant.', modelName='gpt-4o-mini', maxTokens=60, temperature=0.7):
  log.debug('promptTranslationGetResponse started.')
  OPEN_AI_KEY_NAME='OPENAI_API_KEY'
  if not OPEN_AI_KEY_NAME in os.environ:
    log.error(f'Environment variable {OPEN_AI_KEY_NAME} not set.')
    return None
  openaiKey=os.environ[OPEN_AI_KEY_NAME]

  headers = {
      "Authorization": f"Bearer {openaiKey}",
      "Content-Type": "application/json",
  }

  # The data payload with your prompt and other parameters
  data = {
      "model": modelName,
      "messages": [
          {
              "role": "system",
              "content": systemRole
          },
          {
              "role": "user",
              "content": userPrompt
          }
      ],
      "max_tokens": int(maxTokens),
      "temperature": float(temperature),
  }
  log.debug(pprint.pformat(data))

  response= requests.post(UrlCompletions, json=data, headers=headers)  
  if 200!=response.status_code:
    log.warn(f'promptTranslationGetResponse failed. status_code:{response.status_code}.')    
  else:
    log.debug('promptTranslationGetResponse terminated:success.')
  return response

In [7]:
def parseOpenAIResponse( response) -> LlmResponse:
  if 200!=response.status_code:
    log.warn(f'Invalid response. status_code:{response.status_code}')    
    return None  

  responseStruct= response.json()  
  choices= responseStruct['choices']
  if len(choices)<=0:
    log.error('The response has no content')
    return None
  choice= choices[0]    
  content = choice['message']['content']

  usage= responseStruct['usage']

  finishReason= parseFinishReason(choice['finish_reason'])   
  return LlmResponse( content, usage['prompt_tokens'], usage['completion_tokens'], finishReason)   

In [8]:
response= promptTranslationGetResponse("Bye, see you later.",
                                       systemRole='You are a helpful assistant that translates from English to French.')

15:54:32 [DEBUG] - promptTranslationGetResponse started.
15:54:32 [DEBUG] - {'max_tokens': 60,
 'messages': [{'content': 'You are a helpful assistant that translates from '
                          'English to French.',
               'role': 'system'},
              {'content': 'Bye, see you later.', 'role': 'user'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.7}
15:54:32 [DEBUG] - Starting new HTTPS connection (1): api.openai.com:443
15:54:33 [DEBUG] - https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
15:54:33 [DEBUG] - promptTranslationGetResponse terminated:success.


In [9]:
pprint.pp(response.json())

{'id': 'chatcmpl-ArQz3zEILosWwbUCIhG1Ej0J9NYUO',
 'object': 'chat.completion',
 'created': 1737298473,
 'model': 'gpt-4o-mini-2024-07-18',
 'choices': [{'index': 0,
              'message': {'role': 'assistant',
                          'content': 'Au revoir, à plus tard.',
                          'refusal': None},
              'logprobs': None,
              'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 29,
           'completion_tokens': 8,
           'total_tokens': 37,
           'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0},
           'completion_tokens_details': {'reasoning_tokens': 0,
                                         'audio_tokens': 0,
                                         'accepted_prediction_tokens': 0,
                                         'rejected_prediction_tokens': 0}},
 'service_tier': 'default',
 'system_fingerprint': 'fp_72ed7ab54c'}


In [10]:
llmResponse = parseOpenAIResponse( response)

In [11]:
print(llmResponse)

content:"Au revoir, à plus tard."
numPromptTokens:29
numCompletionTokens:8
finishReason:Stop


## Hallucination examples

### Lack of detailed knowledge

In [12]:
response= promptTranslationGetResponse("What is the name of the Towards AI developed largest open-source model and what is its size?")
llmResponse = parseOpenAIResponse( response)
print(llmResponse)
print(llmResponse.content)

15:54:33 [DEBUG] - promptTranslationGetResponse started.
15:54:33 [DEBUG] - {'max_tokens': 60,
 'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
              {'content': 'What is the name of the Towards AI developed '
                          'largest open-source model and what is its size?',
               'role': 'user'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.7}
15:54:33 [DEBUG] - Starting new HTTPS connection (1): api.openai.com:443
15:54:34 [DEBUG] - https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
15:54:34 [DEBUG] - promptTranslationGetResponse terminated:success.


content:"The largest open-source model developed by Towards AI is called **GPT-NEOX**. As of my last knowledge update in October 2023, the largest version of GPT-NEOX is **20 billion parameters**."
numPromptTokens:36
numCompletionTokens:47
finishReason:Stop
The largest open-source model developed by Towards AI is called **GPT-NEOX**. As of my last knowledge update in October 2023, the largest version of GPT-NEOX is **20 billion parameters**.


### Bias: profession gender

In [13]:
response= promptTranslationGetResponse("Translate the following from English to Italian: A Nurse saved the situation yesterday.")
llmResponse = parseOpenAIResponse( response)
print(llmResponse)
print(llmResponse.content)

15:54:34 [DEBUG] - promptTranslationGetResponse started.
15:54:34 [DEBUG] - {'max_tokens': 60,
 'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
              {'content': 'Translate the following from English to Italian: A '
                          'Nurse saved the situation yesterday.',
               'role': 'user'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.7}
15:54:34 [DEBUG] - Starting new HTTPS connection (1): api.openai.com:443
15:54:35 [DEBUG] - https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
15:54:35 [DEBUG] - promptTranslationGetResponse terminated:success.


content:"Una infermiera ha salvato la situazione ieri."
numPromptTokens:32
numCompletionTokens:12
finishReason:Stop
Una infermiera ha salvato la situazione ieri.


In [14]:
response= promptTranslationGetResponse("Translate the following from English to Italian: An engineer saved the situation yesterday.")
llmResponse = parseOpenAIResponse( response)
print(llmResponse)
print(llmResponse.content)

15:54:35 [DEBUG] - promptTranslationGetResponse started.
15:54:35 [DEBUG] - {'max_tokens': 60,
 'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
              {'content': 'Translate the following from English to Italian: An '
                          'engineer saved the situation yesterday.',
               'role': 'user'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.7}
15:54:35 [DEBUG] - Starting new HTTPS connection (1): api.openai.com:443
15:54:36 [DEBUG] - https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
15:54:36 [DEBUG] - promptTranslationGetResponse terminated:success.


content:"Un ingegnere ha salvato la situazione ieri."
numPromptTokens:32
numCompletionTokens:12
finishReason:Stop
Un ingegnere ha salvato la situazione ieri.


### Access to the latest information.

In [15]:
response= promptTranslationGetResponse("What is the date of your more recent available information?")
llmResponse = parseOpenAIResponse( response)
print(llmResponse)
print(llmResponse.content)

15:54:36 [DEBUG] - promptTranslationGetResponse started.
15:54:36 [DEBUG] - {'max_tokens': 60,
 'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
              {'content': 'What is the date of your more recent available '
                          'information?',
               'role': 'user'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.7}
15:54:36 [DEBUG] - Starting new HTTPS connection (1): api.openai.com:443
15:54:37 [DEBUG] - https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
15:54:37 [DEBUG] - promptTranslationGetResponse terminated:success.


content:"My most recent available information is up to October 2023."
numPromptTokens:28
numCompletionTokens:14
finishReason:Stop
My most recent available information is up to October 2023.


In [16]:
response= promptTranslationGetResponse("What is the latest Mission Impossible Movie? The latest Avengers?", maxTokens=100)
llmResponse = parseOpenAIResponse( response)
print(llmResponse)
print(llmResponse.content)

15:54:37 [DEBUG] - promptTranslationGetResponse started.
15:54:37 [DEBUG] - {'max_tokens': 100,
 'messages': [{'content': 'You are a helpful assistant.', 'role': 'system'},
              {'content': 'What is the latest Mission Impossible Movie? The '
                          'latest Avengers?',
               'role': 'user'}],
 'model': 'gpt-4o-mini',
 'temperature': 0.7}
15:54:37 [DEBUG] - Starting new HTTPS connection (1): api.openai.com:443
15:54:40 [DEBUG] - https://api.openai.com:443 "POST /v1/chat/completions HTTP/1.1" 200 None
15:54:40 [DEBUG] - promptTranslationGetResponse terminated:success.


content:"As of October 2023, the latest Mission Impossible movie is "Mission: Impossible – Dead Reckoning Part One," which was released in July 2023. 

The latest Avengers movie is "Avengers: Endgame," which was released in April 2019. However, there are upcoming Avengers films planned, including "Avengers: The Kang Dynasty," scheduled for May 2026, and "Avengers: Secret Wars," set for May 2027."
numPromptTokens:29
numCompletionTokens:96
finishReason:Stop
As of October 2023, the latest Mission Impossible movie is "Mission: Impossible – Dead Reckoning Part One," which was released in July 2023. 

The latest Avengers movie is "Avengers: Endgame," which was released in April 2019. However, there are upcoming Avengers films planned, including "Avengers: The Kang Dynasty," scheduled for May 2026, and "Avengers: Secret Wars," set for May 2027.
